In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import warnings
import pandas as pd
import time
import os
from tqdm.notebook import tqdm
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from collections import Counter
from collections import defaultdict
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import normalize
import math
from konlpy.tag import Twitter
from nltk.tag import pos_tag
from nltk.tag import untag
from nltk import Text
from matplotlib import pyplot as plt
from nltk import FreqDist
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer
from glob import glob
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": "C:/Users/kimEn/Desktop", #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
driver = webdriver.Chrome('chromedriver.exe', options=options)

driver.get('https://paperswithcode.com/sota')

In [3]:
#맨처음 목록 목차 분류작업

class_2b_url=[]  #big 분류페이지로
class0=[] #class_big와 길이확인**

#browse sota대분류 class0 -  중분류 class1  소분류 -pdf설명페이지-pdf다운 길이확인ㅇ
#class0=맨처음페이지 대분류 리스트, class0_url=대분류페이지 이동url
list1=driver.find_elements(By.CLASS_NAME,'col-md-12 > h4 > a')
for i in list1:
    class0.append(i.text)
    class_2b_url.append(i.get_attribute('href'))

In [4]:
#처음목차와 다음 목차 사전 저장& 넘어갈 url수집
dic_bAndm={}
dic_mAnds={}
class_big=[]
class1=[]  #class m과 길이비교**
class_b2m_url=[]
class_small=[]
class_m2s_url=[]
class_small_dic=[]
tmp=[]
mid_url=[]

#대분류url로 이동
for i in range(len(class_2b_url)):
    driver.get(class_2b_url[i]) 
    
#class1=computer vision하위 518개 
#class1=대분류 내 중분류 리스트, class1_url=see all 클릭시 넘어가는 url / none일경우 소분류로 넘어가기 추가**** 
    list1=driver.find_elements(By.CLASS_NAME,'infinite-container.featured-task > div.row > div.col-md-12 > h2')
    list4=driver.find_elements(By.CLASS_NAME, 'col-lg-12 > h1') #dic1&2(대-중분류) key
    list3=driver.find_elements(By.CLASS_NAME,'col-xl-8.card-col.card-col-title > h1') #소분류text
    
    for i in list4:
        class_big.append(i.text)    
        for j in list1:             
            tmp.append(j.text)
            dic_bAndm[i.text]=tmp  #대-중dictionary(list형식)
        tmp=[]
        
#         for k in list3:
#             class_small_dic.append(k.text)
#             dic_mAnds[class1]=class_small_dic   #중-소분류 사전 수정
    
    #mid_url = 중분류에서 바로 소분류로 넘어가는 url
    mid_url_elem=driver.find_elements(By.CLASS_NAME,'card > a')
    for i in mid_url_elem:
        if i not in mid_url:
            mid_url.append(i.get_attribute('href'))
    
    list2 = driver.find_elements(By.CLASS_NAME,'sota-all-tasks > a')
    for i in list2:          
            class_b2m_url.append(i.get_attribute('href'))



In [5]:
#중분류 모두보기 있는 경우 앞과 겹치지 않게 url 수집 mid_url에 저장

for i in range(len(class_b2m_url)):
    driver.get(class_b2m_url[i])
    list4=driver.find_elements(By.CLASS_NAME,'card-title')
    list5=driver.find_elements(By.CLASS_NAME,'card > a')
    
#      #소분류
#     for i in list4:
#         if i not in list4:
#             class_small.append(i.text)
     
    #중분류 모두보기에서 소분류 url 수집
    for i in list5:
        if i not in mid_url:
            mid_url.append(i.get_attribute('href'))



In [5]:
#목록 df저장/csv저장-없는 것만 진행

In [11]:
# 논문 제목, 상세 url 수집
# classname>css section

title = []
url = []
pdf=[]

#mid_url(소분류 url) 반복하면서 title, url 리스트 생성
for n,i in enumerate(mid_url):
    if n<100:
        driver.get(f'{i}')
        

    #title, url
        Tlist=driver.find_elements(By.CLASS_NAME,'col-lg-9.item-content > h1 > a')
        for i in Tlist:
            if i.get_attribute not in url and i.text not in title: 
                title.append(i.text)
                url.append(i.get_attribute('href'))


In [7]:
#pdf_url 수집
for i in range(len(url)):
    
    driver.get(url[i])
    pdf_url=driver.find_elements(By.CLASS_NAME,'badge.badge-light')[0].get_attribute('href')
    
    if pdf_url not in pdf:
        pdf.append(pdf_url)

In [10]:
len(title),len(url),len(pdf)

['https://paperswithcode.com/paper/deep-residual-learning-for-image-recognition',
 'https://paperswithcode.com/paper/very-deep-convolutional-networks-for-large',
 'https://paperswithcode.com/paper/mobilenets-efficient-convolutional-neural',
 'https://paperswithcode.com/paper/densely-connected-convolutional-networks',
 'https://paperswithcode.com/paper/cspnet-a-new-backbone-that-can-enhance',
 'https://paperswithcode.com/paper/mobilenetv2-inverted-residuals-and-linear',
 'https://paperswithcode.com/paper/grad-cam-visual-explanations-from-deep',
 'https://paperswithcode.com/paper/efficientnet-rethinking-model-scaling-for',
 'https://paperswithcode.com/paper/rethinking-the-inception-architecture-for',
 'https://paperswithcode.com/paper/an-image-is-worth-16x16-words-transformers-1',
 'https://paperswithcode.com/paper/distilling-the-knowledge-in-a-neural-network',
 'https://paperswithcode.com/paper/well-read-students-learn-better-the-impact-of',
 'https://paperswithcode.com/paper/distilbert

In [8]:
df=pd.DataFrame(columns=['title','url','pdf'])

df['title']=title
df['url']=url
df['pdf']=pdf

ValueError: Length of values (675) does not match length of index (931)

In [ ]:
#내용저장 csv
df.to_csv("theory.csv",header=True, index=False)
df_theory=pd.read_csv('theory.csv')
df_theory

In [ ]:
#os로 파일 다운로드
pdf_url = df['pdf'].values.tolist()
pdf_file = [i.split('/')[-1] for i in df['pdf'].values if 'arxiv' in i]

print(f'전체 파일 수: {len(pdf_url)}')

#enumerate -> index / for n, ( , ) = n=index, ( , )=tuple
#start /b =>os background실행
for n, (url, file) in enumerate(zip(pdf_url, pdf_file)):
    print(n)
    download_path = f'C:/Users/kimEn/Downloads/pdf_download'
    os.system(f'curl "{url}" --output "{download_path}/{file}"')

In [ ]:
#pdf text변환 / 다른 사이트 참고 ->  출처: https://koreapy.tistory.com/829
def pdf_to_txt(pdf_file):
    rsrcmgr = PDFResourceManager()
   
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(pdf_file, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,
                                 caching=caching, check_extractable=True):
        interpreter.process_page(page)
    
    text = retstr.getvalue()
    
    fp.close()
    device.close()
    retstr.close()
    pdf_txt=[]
    pdf_txt.append(text)
    return text

In [ ]:
#오류제외 파일변환

for i in tqdm(pdf_file):    
    try:
        text = pdf_to_txt(f'{download_path}/{i}')
        txt_file_name = '.'.join(i.split('.')[:-1])
        with open(f'{download_path}/txt/{txt_file_name}.txt', 'w', encoding='utf-8') as f:
            f.write(text)
    except Exception as e:
        print(e, i)

In [ ]:
#다른 사이트 참고 -> 출처:

def get_wordnet_pos(tagged_pos):
    for pos in ['V', 'N', 'J', 'R']:
        if tagged_pos.startswith(pos):
            return pos.lower() if pos != 'J' else 'a'
    return None

lemm = WordNetLemmatizer()

for txt_file in tqdm(glob('./Downloads/pdf_download2/txt/*.txt')):
    with open(txt_file, encoding='utf-8') as f:
        txt = f.read().lower().replace('-\n', '')
        
        
    # 토크나이저
        re_tokenizer = RegexpTokenizer('[a-zA-Z]{2,}')

        # 토큰화
        word_tokens = re_tokenizer.tokenize(txt)

        # 불용어(stopwords) 제거
        stop_words = stopwords.words('english')
        stop_words.append('cid')
        word_tokens = [w for w in word_tokens if w not in stop_words]

        #stemmer = PorterStemmer()
        stemmer = SnowballStemmer('english')
        word_tokens = [stemmer.stem(w) for w in word_tokens]

        word_tokens = pos_tag(word_tokens)
        word_tokens = [(w, get_wordnet_pos(tag)) for w, tag in word_tokens if get_wordnet_pos(tag) != None]
        word_tokens = [lemm.lemmatize(word, pos=tag) for word, tag in word_tokens]



In [ ]:
import os

txt_title=[]
for txt_file in tqdm(glob('./Downloads/pdf_download2/txt/*.txt')):
    with open(txt_file, encoding='utf-8') as f:
        #txt = f.read().lower().replace('-\n', '')
        #sentence=sent_tokenize(txt)
        sentence=f.read().lower().replace('-\n', '').replace('\n', ' ').split('.')
        txt_title.append(txt_file)
        #print(os.system('os.curdir()')) #현재 디렉토리
        print(os.path.basename(txt_file)) #파일명 추출

#sentence가 들어있던 txt_title 불러오기: sentence를 사전으로 저장/


In [ ]:
#빈도수 높은 단어 설명 문장 추출1+출처 표기(논문제목+이름)
#lower 걸기
#fileinput모듈로 대체
#(cid:숫자)=글꼴 이름 제외, 수식 제외, pos tag

import fileinput
import string
import re

meaning={}
#keywords=[x for x in FreqDist(word_tokens) if x.index()<1000]
keywords=FreqDist(word_tokens)
terminology=[]


# with open(glob('./Downloads/pdf_download/txt/*.txt'), encoding='utf-8') as f:
#     # replace_space(f)
#     sentence=f.read().lower().split('.')
for txt_file in tqdm(glob('./Downloads/pdf_download2/txt/*.txt')):
    with open(txt_file, encoding='utf-8') as f:
        #txt = f.read().lower().replace('-\n', '')
        #sentence=sent_tokenize(txt)
        sentence=f.read().lower().replace('-\n', '').replace('\n', ' ').split('.')
        txt_title=txt_file

        
for s in sentence:
    for k in keywords:
        
        #word-token에서 verb만 일치하도록 변경
        #if s.find(k)>0 and ((s.find('is')>0 and t[0] for t in tagged_list if t[1].startswith('V')) or (s.find('are')>0)):  
        
        if k in s and ' is ' in s and 'cid' not in s: 
            if s.isascii()==True:
                tmp=[]
                tmp.append(s)    
                meaning[k]=s
                #print(os.path.basename(txt_file))
            


          
        else:
            pass
        
#         for i in range(len(keywords[0])):
#             if k[0].index(i)
# #         if k[0][i].index() < k[0].index()
#             terminology=[]
print(meaning)


In [ ]:
#{f}가 제일먼저 나오는 text, 출처url
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
research=[]
href=[]

for n,f in enumerate(FreqDist(word_tokens)):
    if n<100:
        driver.get('https://www.google.com')
        elem=driver.find_element(By.CLASS_NAME, 'gLFyf.gsfi')

        # 검색
        elem.clear()
        elem.send_keys(f'what is {f} of deep learning')
        driver.find_element(By.CLASS_NAME, 'gNO89b').submit()                   
        
        first_list=driver.find_elements(By.CLASS_NAME, 'yp1CPe.wDYxhc.NFQFxe.viOShc.LKPcQc')
        href_list=driver.find_elements(By.CLASS_NAME, 'yp1CPe.wDYxhc.NFQFxe.viOShc.LKPcQc > div > div > div > div > div > div > a')
            

        for i in first_list:            
            research.append(i.text)          
            
            print(i.text)
            
        #replace
        
            for i in href_list:           
  
                href.append(i.get_attribute('href'))
                print(i.get_attribute('href'))
    
        time.sleep(5)


In [ ]:
driver.get('https://www.google.com')


for n,f in enumerate(FreqDist(word_tokens)):
    if n<100:
        driver.get('https://www.google.com')
        elem=driver.find_element(By.CLASS_NAME, 'gLFyf.gsfi')

        # 검색
        elem.clear()
        elem.send_keys(f'what is {f} of deep learning')
        driver.find_element(By.CLASS_NAME, 'gNO89b').submit()
#driver.find_element(By.ID, "id_global_search_input").send_keys(Keys.ENTER)



In [ ]:
#스니펫 안나오는것 -> 관련질문

#{f}가 제일먼저 나오는 text, 관련질문 맨 위에 있는 거 가져오기 ->클릭해야 보임
#관련 내용 부분
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize


for n,f in enumerate(FreqDist(word_tokens)):
    if n<100:
        driver.get('https://www.google.com')
        elem=driver.find_element(By.CLASS_NAME, 'gLFyf.gsfi')

        # 검색
        elem.clear()
        elem.send_keys(f'what is {f} of deep learning')
        driver.find_element(By.CLASS_NAME, 'gNO89b').submit()
        
        mlist=driver.find_elements(By.CLASS_NAME, 'Wt5Tfe')
        clist=driver.find_elements(By.CLASS_NAME, 'wQiwMc.ygGdYd.related-question-pair')
        hreflist=driver.find_elements(By.CLASS_NAME, 'yuRUbf > a')


        def click_question():
            for n,c in enumerate(clist):
                if n<5:
                    c.click()
            

        click_question()

        for i in mlist:
             lines=mlist[0].text.split('관련')

        #print(lines)
        research=[]
        research.append(lines[1].replace('질문','').replace('\n',' '))
        print(research)
        # for n,i in enumerate(lines):
        #     print(n,i+'\n')

        #replace
        href=[]
        for i in hreflist:
            href.append(i.get_attribute('href'))

        print(href[1])


In [ ]:
#구글번역기로 번역<>
from googletrans import Translator

Ttranslated={}

# for i in meaning.values():
    
#     translator=Translator()
#     result=translator.translate(i,src='en', dest='ko')
#     #Ttranslated[meaning.keys()]=result
#     print(i)
#     print(result)


translator=Translator()
result=translator.translate(research[0], src='en', dest='ko')
# #Ttranslated[meaning.keys()]=result
print(research[0])
print(result.text)
Ttranslated[research[0]]=result.text #번역 저장 어떻게 할 것인지




In [ ]:
#meaning불러오기 / 없는거 search로 불러오기 / 빈도수 정렬

for n,(x,y) in enumerate(meaning.items()):
    print(n, x+':'+y)

In [ ]:
#csv저장: df: title, url, pdf_url 
#df2: keywords, meaning, translate, meaning_url
#df3: keywords, big_classification, middle_class, small_class


In [ ]:
#(단어-분류)-(의미 번역-원문-출처) -> csv 저장
import pandas as pd
df2=pd.DataFrame(columns=['단어','설명','원문','출처'])

df2['단어']=meaning.keys()
df2['원문']=meaning.values()
df2['설명']=Ttranslated

df2

In [ ]:
#distionary 성능향상